<a style="float:left;" href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Couverture_Terrestre/ImagesSatellitaires-Classification-CouvertureTerrestre.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<br/>
### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter);
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables;
* Pour obtenir de l'information sur une fonction, utilisez la commande Python `help(`"nom de la fonction"`)`

* **SVP**, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

# Images satellitaires - Classification de la couverture terrestre

## Utilisation d'un réseau convolutif

## Inspiration et droits d'auteur

Ce laboratoire s'inspire de plusieurs oeuvres en logiciels libres qui ont été transformées.

Plus précisément, ce carnet IPython pour Google Colab est une traduction et une adaptation du tutoriel « <a href="https://github.com/patrickcgray/open-geo-tutorial/blob/master/notebooks/chapter_6_neural_networks.ipynb" target='_blank'>Chapter 6: Using Neural Networks for Classification and Land Cover Mapping</a> » de <a href="https://github.com/patrickcgray" target='_blank'><b>Patrick Gray</b></a> par <a href="https:/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Couverture_Terrestre/ImagesSatellitaires-AnalyseCouvertureTerrestre.ipynb" target='_blank'>Claude Coulombe</a>.

##### Copyright (c) 2019-2022, Patrick Gray
##### Copyright (c) 2022-2024, Claude Coulombe

## Introduction

L'apprentissage profond est à la fois surexposé sur le plan médiatique et sous-utilisé en pratique (https://doi.org/10.1038/nature14539). Peu importe,  l'apprentissage profond constitue un puissant coffre à outils pour analyser d'énormes quantités de données. De nouvelles applications passionnantes se développent rapidement en <a href="https://doi.org/10.1109/MGRS.2017.2762307" target='_blank'>télédétection</a> dans des domaines tels que la <a href="https://doi.org/10.1109/TGRS.2018.2872509" target='_blank'>détection d'anomalies</a>, la <a href="https://doi.org/10.1109/TGRS.2016.2612821" target='_blank'>classification des images</a> et la <a href="https://doi.org/10.3390/rs11070768" target='_blank'>régression des variables biophysiques</a>.

Vous expérimenterez ici un exemple simple de classification de la couverture terrestre à partir d'images satellitaires de <i>Landsat 8</i> du <i>National Landcover Dataset</i> américain comme données d'entraînement et d'un réseau convolutif profond.

Lancé par la <i>NASA</i> au début des années 1970, <i>Landsat</i> est un programme d’observation de la terre basé sur une constellation de 8 satellites.  <i>Landsat</i> est destiné à des applications civiles (agriculture, utilisation des sols, foresterie, urbanisme, etc.)

## Plan

*   Installation de bibiothèques Python requises;
*   Téléchargement des données depuis GitHub;
*   Création de fonctions utilitaires;
*   Exploration des données;
*   Création d'une base de référence
    * l'algorithme de la forêt aléatoire (<i>random forest</i>);
*   Création et entraînement d'un réseau convolutif;
*   Évaluation et comparaison du modèle.

## Installation de bibliothèques Python

In [ ]:
!export GTIFF_SRS_SOURCE=EPSG
!export TF_XLA_FLAGS="--tf_xla_enable_xla_devices=false"
import os
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices=false'

In [ ]:
# Installation de bibliothèques Python
! pip3 install geopandas rasterio matplotlib descartes scikit-learn

## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>
##### **Note**: Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 42

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)

# os.environ['TF_DETERMINISTIC_OPS'] = '1'
# os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Téléchargement des données depuis GitHub

In [ ]:

!wget https://github.com/ClaudeCoulombe/open-geo-tutorial/blob/master/data.zip?raw=True -O data.zip

### Décompression des données

In [ ]:
import os
import zipfile

fichier_archive = zipfile.ZipFile('data.zip','r')
fichier_archive.extractall('.')
fichier_archive.close()

In [ ]:
!ls -all data

### Les différentes classes de couverture du sol

In [ ]:
# Dictionnaire de toutes les classes et de leur identifiant
classes_couverture = dict((
    (0,  "arrière-plan"), # (0,  'Background'),
    (1,  "non-classé"), # (1, 'Unclassified'),
    # anthropoformé ou aménagé ?
    (2,  "fortement aménagé"), # (2, 'High Intensity Developed'),
    (3,  "moyennement aménagé"), # (3, 'Medium Intensity Developed'),
    (4,  "faiblement aménagé"), # (4, 'Low Intensity Developed'),
    (5,  "espace ouvert aménagé"),# (5, 'Open Space Developed'),
    (6,  "terre cultivée"), # (6, 'Cultivated Land'),
    (7,  "pâturage/foin"), # (7, 'Pasture/Hay')
    (8,  "prairie"), # (8, 'Grassland'),
    (9,  "forêt de feuillus"), # (9, 'Deciduous Forest'),
    (10, "forêt de conifères"), # (10, 'Evergreen Forest'),
    (11, "forêt mixte"), # (11, 'Mixed Forest'),
    (12, "arbuste/broussaille"), # (12, 'Scrub/Shrub'),
    (13, "milieu humide boisé palustre"), # (13, 'Palustrine Forested Wetland'),
    (14, "milieu humide arbustif/broussailleux palustre"), # (14, 'Palustrine Scrub/Shrub Wetland'),
    (15, "milieu humide émergent palustre"), # (15, 'Palustrine Emergent Wetland'),
    (16, "milieu humide boisé estuarien"), # (16, 'Estuarine Forested Wetland'),
    (17, "milieu humide arbustif/broussailleux estuarien"), # (17, 'Estuarine Scrub/Shrub Wetland')
    (18, "milieu humide émergent estuarien"), # (18, 'Estuarine Emergent Wetland'),
    (19, "rivage meuble/non consolidé"), # (19, 'Unconsolidated Shore'),
    (20, "terre nue"), # (20, 'Bare Land'),
    (21, "eau"), # (21, 'Water'),
    (22, "lit de cours d'eau palustre"), # (22, 'Palustrine Aquatic Bed'),
    (23, "lit de cours d'eau estuarien"), # (23, 'Estuarine Aquatic Bed'),
    (24, 'toundra'), # (24, 'Tundra'),
    (25, 'neige/glace') # (25, 'Snow/Ice')
))

print('Code exécuté!')

## Création de fonctions utilitaires

### Un échantillonneur d'images

Échantillonneur aléatoire d'images d'entraînement définis par leurs positions x, y dans une image. L'échantillonneur tient compte des tailles différentes des classes-cibles et fournit un jeu de données d'entraînement équilibré.

Cette fonction extrait un nombre nbre_donnees_entrainement + nbre_donnees_validation aléatoire d'images d'une liste de jeux de données matricielles (<i>raster</i>) fournie en entrée. Elle retourne une liste de positions d'images d'entraînement associée à une image (index_image) et une liste de positions d'images de validation également avec un index d'image.


In [ ]:
import math
import random
import itertools

from rasterio.plot import adjust_band
import matplotlib.pyplot as plt
from rasterio.plot import reshape_as_raster, reshape_as_image
from rasterio.plot import show
from rasterio.windows import Window
import rasterio.features
import rasterio.warp
import rasterio.mask

from pyproj import Proj, transform
from tqdm import tqdm
from shapely.geometry import Polygon

def echantillonner_images(liste_jeux_donnees_landsat,
                  jeu_donnees_etiquettes,
                  etiquettes_couverture,
                  classes_couverture,
                  nombre_donnees_d_entrainement,
                  fusionner=False):

   # Rappel: système de référence géodésique / cartographique, en anglais "Coordinate Reference System" (CRS)
   # Obtenir le système de projection cartographique des étiquettes de classe-cible
   projection_cartographique_etiquettes = Proj(jeu_donnees_etiquettes.crs)
   # initialiser le jeu de pixel d'entraînement
   pixels_entrainement = []
   nbr_donnees_entrainement_par_jeu = math.ceil(nombre_donnees_d_entrainement / len(liste_jeux_donnees_landsat))
   print("Nombre de donnees d'entraînement par jeu de données:",nbr_donnees_entrainement_par_jeu)
   print("Nombre d'images dans le jeu de données:",len(liste_jeux_donnees_landsat))
   for index, jeu_donnees_images in enumerate(liste_jeux_donnees_landsat):
       # Combien de points de données pour chaque classe-cible?
       points_par_classe = nbr_donnees_entrainement_par_jeu // len(np.unique(fusionner_classes(etiquettes_couverture,classes_couverture)[0]))
       print("Nombre de points de données par classe-cible:",points_par_classe)
       # obtenir les limites (4 coins) de l'image Landsat
       # créer un masque approximatif pour le jeu de données dans les coordonnées cartographiques
       # cette fonction transforme les positions des pixels dans les coordonnées (rangée, colonne) en des positions spatiales (x, y)
       points_format_matriciel = jeu_donnees_images.transform * (0, 0),\
                       jeu_donnees_images.transform * (jeu_donnees_images.width, 0),\
                       jeu_donnees_images.transform * (jeu_donnees_images.width, jeu_donnees_images.height),\
                       jeu_donnees_images.transform * (0, jeu_donnees_images.height)
       print("points_format_matriciel:",points_format_matriciel)
       # Obtenir le système de projection cartographique des images
       projection_cartographique_landsat = Proj(jeu_donnees_images.crs)
       neo_points_format_matriciel = []
       # convertir les limites en format matriciel d'images Landsat en coordonnées cartographique des étiquettes
       for x,y in points_format_matriciel:
           x,y = transform(projection_cartographique_landsat,projection_cartographique_etiquettes,x,y)
           # convertir les coordonnées cartographiques x, y en rangées et colonnes du format de l'étiquette
           rangee, colonne = jeu_donnees_etiquettes.index(x, y)
           # puisque rangee, colonne est en fait y, x, donc il faut inverser leur ordre
           neo_points_format_matriciel.append((colonne, rangee))
       # transformer ces nouvelles coordonnées en un polygone
       polygone_matriciel = Polygon(neo_points_format_matriciel)
       # construire une fenêtre à partir de tuples / listes d'index de début et de fin.
       # Window.from_slices((rangee_debut, rangee_fin), (colonne_debut, colonne_fin))
       masque_image_etiquette = jeu_donnees_etiquettes.read(window=Window.from_slices((int(polygone_matriciel.bounds[1]),
                                                                                   int(polygone_matriciel.bounds[3])),
                                                                                  (int(polygone_matriciel.bounds[0]),
                                                                                   int(polygone_matriciel.bounds[2]))))
       if fusionner:
           masque_image_etiquette = fusionner_classes(masque_image_etiquette,classes_couverture)[0]
       # Boucler sur toutes les classes
       tous_les_points_d_une_image = []
       bar_progression = tqdm(np.unique(fusionner_classes(etiquettes_couverture,classes_couverture)[0]))
       for classe_id in (bar_progression):
           bar_progression.set_description("Traitement de la classe-cible « %s »" % str(classes_couverture[int(classe_id)]))
           classe_id = int(classe_id)
           # masquer l'image du sous-ensemble d'étiquettes associée à chaque classe
           # extraire les positions où le masque est vrai
           rangees,colonnes = np.where(masque_image_etiquette[0] == classe_id)
           toutes_les_positions = list(zip(rangees,colonnes))
           # mélanger au hasard toutes les positions
           random.shuffle(toutes_les_positions)
           # convertir dans le système de référence géodésique / cartographique (CRS, Coordinate Reference System)
           # des images satellitaires Landsat
           points_coordonnees_cartographiques_landsat = []
           if len(toutes_les_positions)!=0:
               for r,c in toutes_les_positions[:points_par_classe]:
               # convertir la ligne et la colonne de l'étiquette en coordonnées cartographique de l'étiquette
                   x,y = jeu_donnees_etiquettes.xy(r+polygone_matriciel.bounds[1],c+polygone_matriciel.bounds[0])
               # convertir de la projection cartographique des étiquettes à la projection cartographique Landsat
                   x,y = transform(projection_cartographique_etiquettes,projection_cartographique_landsat,x,y)
               # convertir de l'espace cartographique Landsat en rangées et colonnes
                   r,c = jeu_donnees_images.index(x,y)
                   points_coordonnees_cartographiques_landsat.append((r,c))
               tous_les_points_d_une_image += points_coordonnees_cartographiques_landsat
       liste_index_jeu_de_données = [index] * len(tous_les_points_d_une_image)
       pixels_jeu_de_donnees = list(zip(tous_les_points_d_une_image, liste_index_jeu_de_données))
       pixels_entrainement += pixels_jeu_de_donnees
   random.shuffle(pixels_entrainement)
   return (pixels_entrainement)

print('Code echantillonner_images prêt!')

### Un générateur de tuiles de pixels

Puisque les images satellites sont très grandes, nous devons les diviser en tuiles plus petites.

Le générateur de tuiles prend des positions de pixels et construit des tuiles de pixels au bon format.

Ce générateur est fourni directement au modèle « keras » et alimente en continu les données du modèle pendant l'entraînement et la validation.


In [ ]:
# Générateur de données compatible avec Keras qui génère des tuiles et
# des étiquettes à la volée à partir d'un ensemble de positions de pixels,
# d'un jeu de données d'images satellitaires Landsat à 8 canaux d'images
# et d'un jeu de données d'étiquettes décrivant la couverture terrestre
def generer_tuiles(liste_jeux_donnees_landsat,
                   jeu_donnees_etiquettes,
                   classes_couverture,
                   hauteur_tuile, largeur_tuile,
                   positions_pixel,
                   taille_lot,
                   fusionner=False):

    rangee_pixel = 0
    colonne_pixel = 0
    index_pixel = 0

    # Proj effectue des transformations cartographiques,
    # convertit la longitude, la latitude en coordonnées
    # natives x,y de la projection cartographique et vice versa
    projection_cartographique_etiquettes = Proj(jeu_donnees_etiquettes.crs)

    # On suppose que toutes les images ont le même nombre de bandes spectrales ou canaux
    nombre_canaux = liste_jeux_donnees_landsat[0].count
    compteur_classes = len(classes_couverture)
    tampon = math.ceil(hauteur_tuile / 2)

    while True:
        # nombre_canaux - 1, on enleve un canal (ou bande) parce que nous n'utilisons pas le canal QA
        # qui est la canal de contrôle de la qualité (Quality Assessment) de l'image Landsat
        lot_images = np.zeros((taille_lot, hauteur_tuile, largeur_tuile, nombre_canaux - 1))
        lot_etiquettes = np.zeros((taille_lot,compteur_classes))
        b = 0
        while b < taille_lot:
            # si nous sommes arrivés à la fin des données, il suffit de recommencer
            if index_pixel >= len(positions_pixel):
                index_pixel = 0
            rangee_pixel, colonne_pixel = positions_pixel[index_pixel][0]
            index_jeu_donnees = positions_pixel[index_pixel][1]
            index_pixel += 1
            tuile = liste_jeux_donnees_landsat[index_jeu_donnees].read(list(np.arange(1, nombre_canaux+1)),
                                                                       window=Window(colonne_pixel - tampon,
                                                                                     rangee_pixel - tampon,
                                                                                     largeur_tuile,
                                                                                     hauteur_tuile))
            if tuile.size == 0:
                pass
            elif np.amax(tuile) == 0: # don't include if it is part of the image with no pixels
                pass
            elif np.isnan(tuile).any() == True or -9999 in tuile:
                # nous ne voulons pas de tuiles contenant nan ou -999 qui viennent des bordures
                # cela gaspille du temps et est inefficace
                pass
            elif tuile.shape != (nombre_canaux, largeur_tuile, hauteur_tuile):
                # print('mauvaise dimension')
                # print(tuile.shape)
                # tuile aux mauvaises dimensions
                pass
            elif np.isin(tuile[7,:,:], [352, 368, 392, 416, 432, 480, 840, 864, 880, 904, 928, 944, 1352]).any() == True:
                # le pixel ne doit pas être un nuage
                # cela semble assez inefficace
                # lire: https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/LSDS-1873_US_Landsat_ARD_DFCB_0.pdf
                # print("J'ai trouvé un nuage!")
                # print(tuile[7,:,:])
                pass
            else:
                # retrait de la bande de contrôle de qualité (QA pour Quality Assessment) qui n'est pas utilisée
                tuile = tuile[0:7]
                # reformater du format matriciel (raster) au format d'image normalisé
                tuile_reformatee = (reshape_as_image(tuile)  - 982.5) / 1076.5

                # obtenir l'étiquette de la classe-cible
                # obtenir la géolocalisation du pixel dans l'image
                (x, y) = liste_jeux_donnees_landsat[index_jeu_donnees].xy(rangee_pixel, colonne_pixel)

                # si la projection cartographique des étiquettes est différente
                # convertir la localisation du point à partir duquel nous échantillonnons en la même projection que
                # l'ensemble de données d'étiquettes si nécessaire
                # Rappel: système de référence géodésique / cartographique - Coordinate Reference System (CRS)
                projection_cartographique_landsat = Proj(liste_jeux_donnees_landsat[0].crs)
                # projection_cartographique_images = Proj(jeu_donnees_images.crs)
                if projection_cartographique_landsat != projection_cartographique_etiquettes:
                    x,y = transform(projection_cartographique_landsat,projection_cartographique_etiquettes,x,y)
                # coordonnées de référence de l'étiquette
                rangee, colonne = jeu_donnees_etiquettes.index(x,y)
                # trouver l'étiquette
                # l'image de l'étiquette pourrait être énorme, nous en avons donc besoin pour obtenir une seule position
                fenetre = ((rangee, rangee+1), (colonne, colonne+1))
                donnees, classes_couverture = fusionner_classes(jeu_donnees_etiquettes.read(1,
                                                                                window=fenetre,
                                                                                masked=False,
                                                                                boundless=True),
                                                            classes_couverture)
                etiquette = donnees[0,0]
                # si cette étiquette fait partie d'une couverture non classée alors ignorez la
                if etiquette == 0 or np.isnan(etiquette).any() == True:
                    pass
                else:
                    # ajouter l'étiquette au lot avec un encodage à un bit discriminant (hot encoding)
                    lot_etiquettes[b][etiquette] = 1
                    lot_images[b] = tuile_reformatee
                    b += 1
        yield (lot_images, lot_etiquettes)

print('Code generer_tuiles prêt!')

### Une fonction pour fusionner les classes en un sous-ensemble plus petit

In [ ]:
def fusionner_classes(y,classes_couverture):

    # reclasser 255 (pixel blanc intense) à 0 (arrière-plan)
    y[y == 255] = 0

    # regrouper "moyennement aménagé" 3 et "faiblement aménagé" 4
    # dans "milieu aménagé" 2
    y[y == 3] = 2
    y[y == 4] = 2
    classes_couverture[2] = "milieu aménagé"

    # regrouper "espace ouvert aménagé" 5 et "pâturage/foin" 7
    # dans "terre cultivée" 6
    y[y == 5] = 6
    y[y == 7] = 6
    classes_couverture[6] = "terre cultivée"

    # regrouper "forêt de feuillus" 9 et "forêt de conifères" 10,
    # "arbuste/broussaille" 12, et "milieu humide boisé palustre" 13
    # dans "milieu forestier" 11
    y[y == 9] = 11
    y[y == 10] = 11
    y[y == 12] = 11
    y[y == 13] = 11
    classes_couverture[11] = "milieu forestier"

    # regrouper "milieu humide arbustif/broussailleux palustre" 14
    # "milieu humide émergent palustre" 15, "milieu humide boisé estuarien") 16
    # et "milieu humide arbustif/broussailleux estuarien" 17,
    # dans "milieu humide" 18
    y[y == 14] = 18
    y[y == 15] = 18
    y[y == 16] = 18
    y[y == 17] = 18
    classes_couverture[18] = "milieu humide"

    # regrouper "rivage meuble/non consolidé" 19 et "lit de cours d'eau palustre" 22
    # dans "milieu aquatique" 21
    y[y == 22] = 21
    y[y == 19] = 21
    classes_couverture[21] = "milieu aquatique"

    return(y,classes_couverture)

print('Code fusionner_classes prêt!')

### Matrice de confusion

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
import matplotlib.pyplot as plt

def afficher_matrice_confusion(vraies_etiquettes, etiquettes_predites, classes, dictionnaire_classes,
                               normaliser=False,
                               titre=None,
                               cmap=plt.cm.Blues):
    """
    Cette fonction affiche la matrice de confusion.
    Une normalisation peut être appliquée au besoin
    """
    if not titre:
        if normaliser:
            titre = 'Matrice de confusion normalisée'
        else:
            titre = 'Matrice de confusion sans normalisation'

    # Construction de la matrice de confusion
    cm = confusion_matrix(vraies_etiquettes, etiquettes_predites)
    # Utilisez une seule fois les étiquettes qui apparaissent dans les données
    classes = classes[unique_labels(vraies_etiquettes, etiquettes_predites)]
    # convertir l'identifiant de classe en nom de classe en utilisant un dictionnaire
    nom_classes_pour_affichage = []
    for nom_classe_affichage in classes:
        nom_classes_pour_affichage.append(dictionnaire_classes[nom_classe_affichage])
    if normaliser:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    else:
        pass

    fig, axe = plt.subplots(figsize=(10,10))
    image = axe.imshow(cm, interpolation='nearest', cmap=cmap)
    axe.figure.colorbar(image, ax=axe)
    # montrer touts les graduations (ticks)
    axe.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # étiqueter les graduations avec les noms de classes appropriés
           xticklabels=nom_classes_pour_affichage, yticklabels=nom_classes_pour_affichage,
           title=titre,
           ylabel='Étiquette vraie',
           xlabel='Étiquette prédite')

    # faire pivoter les étiquettes et définir leur alignement.
    plt.setp(axe.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # boucler sur les composants de la matrice de confusion et ajouter des annotations
    fmt = '.2f' if normaliser else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            axe.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return axe

print('Code afficher_matrice_confusion prêt!')

## Exploration des données du satellite Landsat

### Lecture des métadonnées sur les images

In [ ]:
import rasterio
from pyproj import CRS
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np

# Nous allons modifier l'encodage CRS (Coordinate Reference System) de nos images pour EPSG 4326
rasterio.Env(GTIFF_SRS_SOURCE="EPSG")
gdal.SetConfigOption("GTIFF_SRS_SOURCE", "EPSG")

donnees_landsat = gdal.Open("data/landsat_image.tif")

print("Pilote: {}/{}".format(donnees_landsat.GetDriver().ShortName,
                           donnees_landsat.GetDriver().LongName))
print()
print("Taille: {} x {} x {}".format(donnees_landsat.RasterXSize,
                                   donnees_landsat.RasterYSize,
                                   donnees_landsat.RasterCount))
print()
print("Projection: {}".format(donnees_landsat.GetProjection()))

# PROJCS["WGS_1984_Albers",
#        GEOGCS["WGS 84",
#               DATUM["WGS_1984",
#                     SPHEROID["WGS 84",
#                              6378140,298.256999999996,
#                              AUTHORITY["EPSG","7030"]],
#                     AUTHORITY["EPSG","6326"]],
#               PRIMEM["Greenwich",0],
#               UNIT["degree",0.0174532925199433,
#                    AUTHORITY["EPSG","9122"]],
#               AUTHORITY["EPSG","4326"]],
#        PROJECTION["Albers_Conic_Equal_Area"],
#        PARAMETER["latitude_of_center",23],
#        PARAMETER["longitude_of_center",-96],
#        PARAMETER["standard_parallel_1",29.5],
#        PARAMETER["standard_parallel_2",45.5],
#        PARAMETER["false_easting",0],
#        PARAMETER["false_northing",0],
#        UNIT["metre",1,
#             AUTHORITY["EPSG","9001"]],
#        AXIS["Easting",
#             EAST],
#        AXIS["Northing",NORTH]
#        ]

geotransform = donnees_landsat.GetGeoTransform()
if geotransform:
   print()
   print("Origine: ({}, {})".format(geotransform[0], geotransform[3]))
   print()
   print("Taille du pixel: ({}, {})".format(geotransform[1], geotransform[5]))

# Lecture des métadonnées du fichier de données matricielles (raster) Landsat
meta_donnees_images_landsat = rasterio.open("data/landsat_image.tif")

# Vérification du nombre de canaux / bandes spectrales par image
nombre_canaux = meta_donnees_images_landsat.count
print()
print("Nombre de canaux par image: {nc}\n".format(nc=nombre_canaux))

# Combien de rangées et de colonnes dans les données?
rangees, colonnes = meta_donnees_images_landsat.shape
print()
print("Le format des images est: {l} rangees x {c} colonnes\n".format(l=rangees, c=colonnes))

# Quelle est la version de la bibliothèque Rasterio:
print()
print("Version de Rasterio: {v}\n".format(v=rasterio.__version__))

# Quel pilote (driver) a été utilisé pour lire les données matricielles (raster)?
pilote = meta_donnees_images_landsat.driver
print()
print("Pilote données matricielles utilisé: {p}\n".format(p=pilote))

# Quelle est la projection cartographique des données?
projection_cartographique = meta_donnees_images_landsat.crs
print()
print("Projection cartographique utilisée:\n",projection_cartographique)

# Pour éviter un grand nombre d'avertissements (warning) comme ci-dessous

# WARNING:rasterio._env:CPLE_AppDefined in The definition of geographic CRS EPSG:4326 got from GeoTIFF keys
# is not the same as the one from the EPSG registry, which may cause issues during reprojection operations.
# Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys),
# or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.

print()
print("\nModification de l'encodage CRS de nos métadonnées de nos images...")
with rasterio.open("data/landsat_image.tif", "r+") as meta_donnees_images_landsat:
   # Créer un nouvel objet CRS avec l'encodage EPSG
   nouveau_crs = rasterio.crs.CRS.from_epsg(4326)
   # Mettre à jour l'encodage CRS des données
   meta_donnees_images_landsat.crs = nouveau_crs
   # Lire et réécrire chaque bande pour mettre à jour les métadonnées en place
   for i in range(1, meta_donnees_images_landsat.count + 1):
     metadonnees_de_bande = meta_donnees_images_landsat.read(i)
     meta_donnees_images_landsat.write(metadonnees_de_bande, i)

# Relire le fichier de données
meta_donnees_images_landsat = rasterio.open("data/landsat_image.tif", "r+",GTIFF_SRS_SOURCE="EPSG")

###  Lecture et chargement en mémoire des images
ou données matricielles (raster), images satellitaires Landsat


In [ ]:
images_landsat = meta_donnees_images_landsat.read()
images_landsat.shape

### Calcul de l'indice de végétation

Maintenant calculons l'indice de végétation<sup>1</sup>, un indicateur utilisé en télédétection pour évaluer si la cible observée contient de la végétation. L'indice de végétation se calcule à partir des réflectances mesurées dans les bandes visible rouge et le proche infrarouge.
<hr/>
<span style="font-size:80%"><sup>1</sup><b>Note - terminologie:</b> En français,  « indice de végétation par différence normalisée (IVDN) » ou « indice différentiel normalisé de végétation » ou plus simplement « indice de végétation ». En anglais, « <i>Normalized Difference Vegetation Index</i> (<i>NDVI</i>) ».</span>

In [ ]:
bande_proche_infrarouge = images_landsat[4, :, :]
bande_rouge = images_landsat[3, :, :]

indice_vegetation = np.clip((bande_proche_infrarouge.astype(float) - bande_rouge.astype(float)) / (bande_proche_infrarouge.astype(float) + bande_rouge.astype(float)), -1,1)

print('Code exécuté!')

In [ ]:
print('\nIndice de végétation maximum: {m:.2f}'.format(m=indice_vegetation.max()))
print('Indice de végétation moyen: {m:.2f}'.format(m=indice_vegetation.mean()))
print('Indice de végétation médian: {m:.2f}'.format(m=np.median(indice_vegetation)))
print('Indice de végétation minimum: {m:.2f}'.format(m=indice_vegetation.min()))

Examinons la répartition statistique de l'indice de végétation avec un histogramme.

In [ ]:
figure, axes = plt.subplots(figsize=(1.62*6,6))
# Nous pouvons définir le nombre de colonnes de l'histogramme avec l'argument `bins`
axes.hist(indice_vegetation.flatten(), bins=50)
plt.title("Histogramme de l'indice de végétation")
plt.xlabel("Indice de végétation")
plt.ylabel("Fréquence")
plt.show()

### Histogramme multicanal de l'image
L'indice de végétation semble normal, regardons maintenant l'histogramme de l'image dans son intégralité.

Qu'est-ce que la valeur numérique (Digital Number, DN)? Dans les systèmes de télédétection, la la valeur numérique est une valeur attribuée à un pixel, généralement sous la forme d'un entier compris entre 0 et 255 (c'est-à-dire un octet).

In [ ]:
figure, axes = plt.subplots(figsize=(1.62*6,6))

rasterio.plot.show_hist(meta_donnees_images_landsat.read([1,2,3,4,5,6,7]),
                        bins=100,
                        histtype='stepfilled',
                        lw=0.0,
                        stacked=False,
                        alpha=0.3,
                        ax = axes,
                       title="Histogramme multicanal de l'image",
                       label=[1,2,3,4,5,6,7]
                       )
axes.set_xlabel('Valeur numérique des pixels')
_ = axes.set_ylabel('Fréquence')


###  Lecture et chargement en mémoire des étiquettes ou annotation de couverture

In [ ]:
# Pour éviter un grand nombre d'avertissements (warning) comme ci-dessous

# WARNING:rasterio._env:CPLE_AppDefined in The definition of geographic CRS EPSG:4326 got from GeoTIFF keys
# is not the same as the one from the EPSG registry, which may cause issues during reprojection operations.
# Set GTIFF_SRS_SOURCE configuration option to EPSG to use official parameters (overriding the ones from GeoTIFF keys),
# or to GEOKEYS to use custom values from GeoTIFF keys and drop the EPSG code.

# Nous allons modifier l'encodage CRS (Coordinate Reference System) de nos métadonnées d'étiquettes pour EPSG 4326
import rasterio
rasterio.env.GTIFF_SRS_SOURCE = 'EPSG'
from osgeo import gdal
gdal.SetConfigOption("GTIFF_SRS_SOURCE", "EPSG")

print()
print("Modification de l'encodage CRS de nos métadonnées d'étiquettes...")
# Ouvrir en mode lecture & écriture
with rasterio.open("data/labels_image.tif", "r+") as meta_donnees_etiquettes_couverture:
    # Créer un nouvel objet CRS avec un encodage EPSG
    nouveau_crs = rasterio.crs.CRS.from_epsg(4326)
    # Mettre à jour l'encodage CRS des données
    meta_donnees_etiquettes_couverture.crs = nouveau_crs
    # Lire et réécrire chaque bande pour mettre à jour les métadonnées en place
    for i in range(1, meta_donnees_etiquettes_couverture.count + 1):
      donnees_de_bande = meta_donnees_etiquettes_couverture.read(i)
      meta_donnees_etiquettes_couverture.write(donnees_de_bande, i)

# Relire le fichier de métadonnées
meta_donnees_etiquettes_couverture = rasterio.open('data/labels_image.tif')

In [ ]:
# Nous fusionnons des classes pour limiter le nombre de classes avec lesquelles nous travaillons
etiquettes_couverture, classes_couverture = fusionner_classes(meta_donnees_etiquettes_couverture.read(),classes_couverture)
etiquettes_couverture.shape

### Visualisation - Image Landsat, couverture terrestre et indice de végétation

Maintenant, nous allons visualiser l'image Landsat, une carte en fausses couleurs des de la couverture terrestre et l'indice de végétation (NDVI) côte à côte :


In [ ]:
from rasterio.plot import adjust_band
from rasterio.plot import reshape_as_raster, reshape_as_image
from rasterio.plot import show

# extraire les bandes à visualiser
index = np.array([3, 2, 1])
couleurs = images_landsat[index, :, :].astype(np.float64)

# formater l'image Landsat en fonction de l'histogramme ci-dessus
max_val = 2500
min_val = 0

# borner les valeurs maximales et minimales
couleurs[couleurs[:, :, :] > max_val] = max_val
couleurs[couleurs[:, :, :] < min_val] = min_val

for b in range(couleurs.shape[0]):
    couleurs[b, :, :] = couleurs[b, :, :] * 1 / (max_val - min_val)

# Les images matricielles sont dans le format [canaux, rangees, colonnes]
# alors qu'en général les images sont typiqueent en format [rangees, colonnes, canaux]
# et donc notre tableau doit être reformaté
print(couleurs.shape)
couleurs_reformatees = reshape_as_image(couleurs)
print(couleurs_reformatees.shape)

# Configuration d'une palette de couleurs pour les cartes de terrain
fausses_couleurs = dict((
    (0, (245,245,245, 255)), # arrière-plan # Background
    (1, (0,0,0)), # non-classé # Unclassified (Cloud, Shadow, etc)
    (2, (255,0,0)), # fortement aménagé # High Intensity Developed
    (3, (255, 110, 51)), # moyennement aménagé # Medium Intensity Developed
    (4, (255, 162, 51)), # faiblement aménagé # Low Intensity Developed
    (5, (255, 162, 51)), # espace ouvert aménagé # Open Space Developed
    (6, (162, 89, 0)), # terre cultivée # Cultivated Land
    (7, (229, 221, 50)), # pâturage/foin # Pasture/Hay
    (8, (185, 251, 96)), # prairie # Grassland
    (9, (83, 144, 0)), # forêt de feuillus # Deciduous Forest
    (10, (13, 118, 0  )), # forêt de conifères # Evergreen Forest
    (11, (62, 178, 49)), # forêt mixte / Mixed Forest
    (12, (100, 241, 125)), # arbuste/broussaille # Scrub/Shrub
    (13, (68, 160, 85)), # milieu humide boisé palustre # Palustrine Forested Wetland
    (14, (118, 192, 131)), # milieu humide arbustif/broussailleux palustre # Palustrine Scrub/Shrub Wetland
    (15, (188, 0, 211)), # milieu humide émergent palustre # Palustrine Emergent Wetland
    (16, (188, 0, 211)), # milieu humide boisé estuarien # Estuarine Forested Wetland
    (17, (0, 0, 0)), # milieu humide arbustif/broussailleux estuarien # Estuarine Scrub/Shrub Wetland
    (18, (172, 0, 191)), # milieu humide émergent estuarien # Estuarine Emergent Wetland
    (19, (159, 251, 255)), # rivage meuble/non consolidé # Unconsolidated Shore
    (20, (172, 177, 68)), # terre nue # Bare Land
    (21, (29, 0, 189)), # eau # Water
    (22, (40, 40, 40)), # lit de cours d'eau palustre # Pal Bed
))

n = int(np.max(etiquettes_couverture)) + 1

# Normalisation, 0 à 255 devient 0 à 1
for index_fausse_couleur in fausses_couleurs:
    code_fausse_couleur = fausses_couleurs[index_fausse_couleur]
    code_fausse_couleur_normalise = [element_code / 255.0 for element_code in code_fausse_couleur]
    fausses_couleurs[index_fausse_couleur] = code_fausse_couleur_normalise

index_fausses_couleurs = [fausses_couleurs[index] for index in range(0, n)]

cmap_fausses_couleurs = plt.matplotlib.colors.ListedColormap(index_fausses_couleurs,
                                                             'Classification',
                                                             n)
fig, axes = plt.subplots(1, 3, figsize=(1.62*17, 17))

# Afficher l'image en couleur
axes[0].imshow(couleurs_reformatees)
axes[0].set_title('Image Landsat couleur')

# Afficher les classes de couverture en fausses couleurs
axes[1].imshow(etiquettes_couverture[0,:, :],
               cmap=cmap_fausses_couleurs,
               interpolation='none')
import matplotlib.patches as mpatches
items_legende =[mpatches.Patch(color=cmap_fausses_couleurs.colors[classe_id],label=classes_couverture[classe_id])\
                for classe_id in range(len(cmap_fausses_couleurs.colors))]
axes[1].set_title('Classes\nde couverture')
axes[1].legend(handles=items_legende, loc=(1.01,0), borderaxespad=0.)

# Afficher l'indice de végétation
axes[2].imshow(indice_vegetation,
              cmap='RdYlGn')
axes[2].set_title('Indice de végégation')

# Afficher les images
plt.show()


### <b>Attention!</b>

La carte de l'indice de végétation à droite, peut être très utile aux agriculteurs pour la surveillance de leurs champs. En effet, cette carte permet d'identifier du haut des airs les parcelles où la végétation a du mal à pousser en raison du manque d'eau, du manque d'engrais ou de l'action de ravageurs.

Combien y a-t-il de pixels dans chaque classe ?

In [ ]:
unique, frequences = np.unique(etiquettes_couverture, return_counts=True)
donnees_histogramme = [(classes_couverture[classe_id],frequences) for classe_id,frequences in list(zip(unique, frequences))]
donnees_histogramme

In [ ]:
fig = plt.figure(figsize=(1.62*6, 6))
plt.yscale('log')
plt.xticks(rotation=45)
plt.xlabel("Type de couverture")
plt.ylabel("Log(Fréquence)")
plt.title("Répartition des types de couverture terrestre - échelle logarithmique")
_ = plt.bar([classe[0] for classe in donnees_histogramme], [classe[1] for classe in donnees_histogramme])


### Génération des données d'entraînement

In [ ]:
echantillon_images = echantillonner_images([meta_donnees_images_landsat],
                                            meta_donnees_etiquettes_couverture,
                                            etiquettes_couverture,
                                            classes_couverture,
                                            3000,
                                            True)

### Test du générateur de tuiles

Afficher des lots d'images et d'étiquettes et vérifier leurs dimensions.

In [ ]:
lot_images = None
frequence = 0

def normalize(image):
   return (image - np.min(image)) / (np.max(image) - np.min(image))

for (images,etiquettes) in generer_tuiles([meta_donnees_images_landsat],
                                          meta_donnees_etiquettes_couverture,
                                          classes_couverture,
                                          128, 128,
                                          echantillon_images,
                                          10):
   # Arrêter après 3 lots d'images
   if frequence > 2:
       break
   print("Format du lot d'images")
   print(images.shape)
   print("Format du lot d'étiquettes")
   print(etiquettes.shape)
   fig, axes = plt.subplots(10, 7, figsize=(1.62*10, 10))
   for index_lot in range(10):
       for canal in range(7):
           image = images[index_lot,:,:,canal]
           # plt.imshow(normalize(image))
           axes[index_lot,canal].imshow(normalize(image))
   plt.show()
   print('----')
   frequence += 1
   lot_images =  images
   lot_etiquettes = etiquettes

### Visualisation de tuiles

Maintenant, visualisons des tuiles réelles.

In [ ]:
import numpy as np

def normalize(image):
    return (image - np.min(image)) / (np.max(image) - np.min(image))

fig, axes = plt.subplots(2, 3, figsize=(1.62*10, 10))
axes[0,0].imshow(normalize(lot_images[0,:,:,3:6]))
axes[0,0].set_title(classes_couverture[np.argmax(lot_etiquettes[0])])
axes[0,1].imshow(normalize(lot_images[1,:,:,3:6]))
axes[0,1].set_title(classes_couverture[np.argmax(lot_etiquettes[1])])
axes[0,2].imshow(normalize(lot_images[2,:,:,3:6]))
axes[0,2].set_title(classes_couverture[np.argmax(lot_etiquettes[2])])
axes[1,0].imshow(normalize(lot_images[3,:,:,3:6]))
axes[1,0].set_title(classes_couverture[np.argmax(lot_etiquettes[3])])
axes[1,1].imshow(normalize(lot_images[4,:,:,3:6]))
axes[1,1].set_title(classes_couverture[np.argmax(lot_etiquettes[4])])
axes[1,2].imshow(normalize(lot_images[5,:,:,3:6]))
axes[1,2].set_title(classes_couverture[np.argmax(lot_etiquettes[5])])

plt.show()


### Générer un jeu de données d'entraînement de tuiles 1x1 pour scikit-learn


In [ ]:
lot_images = None
lot_etiquettes = None
taille_echantillon = 500

frequence = 0

for (images, etiquettes) in generer_tuiles([meta_donnees_images_landsat],
                                         meta_donnees_etiquettes_couverture,
                                         classes_couverture,
                                         1, 1,
                                         echantillon_images,
                                         taille_echantillon):
    if frequence > 0:
        break
    print("Format du lot d'images")
    print(images.shape)
    print("Format du lot d'étiquettes")
    print(etiquettes.shape)
    print('----')
    frequence += 1
    lot_images =  images
    lot_etiquettes = etiquettes

#### Reformater les données
Reformater les données pour scikit-learn qui a besoin de données au format `(echantillons, bandes)` :

In [ ]:
lot_images[0,:,:,:]

In [ ]:
lot_images_reformatees = lot_images.reshape(taille_echantillon,7)
lot_images_reformatees[0]

### Visualiser les signatures spectrales

Examinons le spectre des intensités pour les différentes bandes ou canaux. Rappelons que la réflectance est le rapport de l'intensité d'une onde réfléchie sur l'intensité de l'onde incidente.

In [ ]:
fig, axe = plt.subplots(1,1, figsize=[1.62*8,8])

liste_couleurs = ['red', # 'milieu aménagé' 2
                  'gold', # 'terre cultivée' 6
                  'orange', # 'prairie' 8
                  'forestgreen', # 'milieu forestier' 11
                  'skyblue', # 'milieu humide' 18
                  'brown', # 'terre nue' 20
                  'blue'] # eau 21

# numbers 1-8
nombre_canaux = np.arange(1,8)

etiquettes = np.argmax(lot_etiquettes, axis=1)
images = lot_images_reformatees

classes = np.unique(etiquettes)
for index_couleur,id_classe in enumerate(classes):
    intensite_canal = np.mean(images[etiquettes==id_classe, :], axis=0)
    # afficher sous forme de lignes tracées dans un graphique
    axe.plot(nombre_canaux,
             intensite_canal,
             color=liste_couleurs[index_couleur],
             label=classes_couverture[id_classe])

# ajouter le nom des axes du graphique
axe.set_xlabel('Bande #')
axe.set_ylabel('Valeur de réflectance')

# ajouter un titre au graphique
axe.set_title("Spectre d'intensité des différentes bandes")
_ = axe.legend(loc='upper left')

## Création d'une base de référence pour la classification

### Générer un ensemble de données d'entraînement de tuiles 1x1 pour scikit-learn

In [ ]:
lot_images = None
lot_etiquettes = None

taille_echantillon = 800
nombre_donnees_d_entrainement = 600

frequence = 0

for (images,etiquettes) in generer_tuiles([meta_donnees_images_landsat],
                                          meta_donnees_etiquettes_couverture,
                                          classes_couverture,1, 1,
                                          echantillon_images,
                                          taille_echantillon):
    if frequence > 0:
        break
    print("Format du lot d'images")
    print(images.shape)
    print("Format du lot d'étiquettes")
    print(etiquettes.shape)
    print('----')
    frequence += 1
    lot_images =  images
    lot_etiquettes = etiquettes

lot_images_reformatees = lot_images.reshape(taille_echantillon,7)

images_entrainement = lot_images_reformatees[:nombre_donnees_d_entrainement]
images_validation = lot_images_reformatees[nombre_donnees_d_entrainement:]
etiquettes_entrainemnet = np.argmax(lot_etiquettes, axis=1)[:nombre_donnees_d_entrainement]
etiquettes_validation = np.argmax(lot_etiquettes, axis=1)[nombre_donnees_d_entrainement:]

### Algorithme de la forêt aléatoire (<i>random forest</i>)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialiser notre modèle avec 500 arbres
foret_aleatoire = RandomForestClassifier(n_estimators=500,
                                         oob_score=True)

# Entraîner le modèle sur les données d'entraînement
foret_aleatoire = foret_aleatoire.fit(images_entrainement, etiquettes_entrainemnet)

print('Exactitude de: {accuracy:.2f}%'.format(accuracy = foret_aleatoire.score(images_validation, etiquettes_validation)*100))

In [ ]:
pred_index = foret_aleatoire.predict(images_validation)

# Plot non-normalized confusion matrix
_ = afficher_matrice_confusion(etiquettes_validation,
                               pred_index,
                               classes=np.array(list(classes_couverture)),
                               dictionnaire_classes=classes_couverture)


In [ ]:
# Plot normalized confusion matrix
_ = afficher_matrice_confusion(etiquettes_validation,
                               pred_index,
                               classes=np.array(list(classes_couverture)),
                               dictionnaire_classes=classes_couverture,
                               normaliser=True)


In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
print("Exactitude: {exactitude:.2f}%".format(exactitude = accuracy_score(etiquettes_validation, pred_index)*100))
print("Précision: {precision:.2f}%".format(precision = precision_score(etiquettes_validation, pred_index, average='weighted')*100))
print("Rappel: {rappel:.2f}%".format(rappel = recall_score(etiquettes_validation, pred_index, average='weighted')*100))
print("Métrique F1: {f1:.2f}%".format(f1 = f1_score(etiquettes_validation, pred_index, average='weighted')*100))


Ce modèle n'est pas terrible, car il classe mal une bonne partie des prairies, des terres cultivées et des terres aménagées. Voyons si nous pouvons améliorer ls résultats avec un réseau convolutif profond...

## Création et entraînement d'un réseau convolutif

### Importer les bibliothèques `keras` nécessaires

In [ ]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Activation, BatchNormalization
from keras.callbacks import ModelCheckpoint
print("TensorFlow version:",tf.__version__)
print("Keras version:",keras.__version__)
print("Bibliothèques Tensorflow et Keras importées!")

### Définir les hyperparamètres du réseau convolutif

In [ ]:
taille_lot = 25
nombre_iterations = 50
nombre_classes = len(classes_couverture)

# input image dimensions
taille_tuile = 32
nombre_rangees_image, nombre_colonnes_image = taille_tuile, taille_tuile
nombre_canaux = meta_donnees_images_landsat.count- 1

format_donnees_entree = (nombre_rangees_image, nombre_colonnes_image, nombre_canaux)
print(format_donnees_entree)

### Créer l'architecture du réseau convolutif

In [ ]:
modele = Sequential()

modele.add(Conv2D(32, (3, 3), padding='same', input_shape=format_donnees_entree))
modele.add(BatchNormalization())
modele.add(Activation('relu'))
# modele.add(MaxPooling2D(pool_size=(2, 2)))
modele.add(AveragePooling2D(pool_size=(2, 2)))

modele.add(Conv2D(64, (3, 3), padding='same'))
modele.add(BatchNormalization())
modele.add(Activation('relu'))

modele.add(Conv2D(64, (3, 3), padding='same'))
modele.add(BatchNormalization())
modele.add(Activation('relu'))
# modele.add(MaxPooling2D(pool_size=(2, 2)))
modele.add(AveragePooling2D(pool_size=(2, 2)))

modele.add(Conv2D(128, (3, 3), padding='same'))
modele.add(BatchNormalization())
modele.add(Activation('relu'))
# modele.add(MaxPooling2D(pool_size=(2, 2)))
modele.add(AveragePooling2D(pool_size=(2, 2)))

modele.add(Conv2D(256, (3, 3), padding='same'))
modele.add(BatchNormalization())
modele.add(Activation('relu'))
# modele.add(MaxPooling2D(pool_size=(2, 2)))
modele.add(AveragePooling2D(pool_size=(2, 2)))
modele.add(Dropout(0.25))

modele.add(Flatten())
modele.add(Dense(128))
modele.add(BatchNormalization())
modele.add(Activation('relu'))
modele.add(Dropout(0.25))

modele.add(Dense(128))
modele.add(BatchNormalization())
modele.add(Activation('relu'))
modele.add(Dropout(0.25))

modele.add(Dense(nombre_classes))
modele.add(Activation('softmax'))

modele.summary()


### Choisir la fonction d'optimisation et compiler le modèle

In [ ]:
planificateur_taux_apprentissage = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)

optimiseur = tf.keras.optimizers.Adam(learning_rate=planificateur_taux_apprentissage)
metrics=['accuracy']

modele.compile(optimizer=optimiseur,
               loss='categorical_crossentropy',
               jit_compile=False,
               metrics=metrics)

print("Modèle compilé")

### Diviser les données entre donnés d'entraînement et données de validation

In [ ]:
ratio_entrainement_vs_validation = 0.8
pixels_validation = echantillon_images[int(len(echantillon_images)*ratio_entrainement_vs_validation):]
echantillon_images = echantillon_images[:int(len(echantillon_images)*ratio_entrainement_vs_validation)]
print("Nombre d'exemples d'entraînement: {n_entrainement} \nNombre d'exemples de validation: {n_validation}".
      format(n_entrainement=len(echantillon_images),n_validation=len(pixels_validation)))

### Entraîner le modèle

In [ ]:
traces = modele.fit(generer_tuiles([meta_donnees_images_landsat],
                                   meta_donnees_etiquettes_couverture,
                                   classes_couverture,
                                   taille_tuile, taille_tuile,
                                   echantillon_images,
                                   taille_lot,
                                   fusionner=True),
                    steps_per_epoch=len(echantillon_images) // taille_lot,
                    epochs=nombre_iterations,
                    verbose=1,
                    validation_data=generer_tuiles([meta_donnees_images_landsat],
                                                   meta_donnees_etiquettes_couverture,
                                                   classes_couverture,
                                                   taille_tuile, taille_tuile,
                                                   pixels_validation,
                                                   taille_lot,
                                                   fusionner=True),
                    validation_steps=len(pixels_validation) // taille_lot)

### Affichage des courbes d'entraînement et de validation

In [ ]:
plt.figure(figsize=(1.62*6,6))
plt.plot(traces.history['accuracy'])
plt.plot(traces.history['val_accuracy'])
plt.title("Courbes d'exactitude du modèle")
plt.ylabel("Exactitude (%)")
plt.xlabel("Nombre d'itérations / époques")
_ = plt.legend(['Entraînement', 'Validation'], loc='upper left')

In [ ]:
plt.figure(figsize=(1.62*6,6))
plt.plot(traces.history['loss'])
plt.plot(traces.history['val_loss'])
plt.title("Courbes d'erreur d'entropie du modèle")
plt.ylabel("Erreur")
plt.xlabel("Nombre d'itérations / époques")
_ = plt.legend(['Entraînement', 'Validation'], loc='upper left')

### Génération de données test

In [ ]:
predictions = modele.predict(generer_tuiles([meta_donnees_images_landsat],
                                            meta_donnees_etiquettes_couverture,
                                            classes_couverture,
                                            taille_tuile, taille_tuile,
                                            pixels_validation,
                                            taille_lot,
                                            fusionner=True),
                             steps=len(pixels_validation) // taille_lot,
                             verbose=1)

eval_generator = generer_tuiles([meta_donnees_images_landsat],
                                meta_donnees_etiquettes_couverture,
                                classes_couverture,
                                taille_tuile, taille_tuile,
                                pixels_validation,
                                1,
                                fusionner=True)

labels = np.empty(predictions.shape)
frequence = 0
while frequence < len(labels):
    image_b, label_b = next(eval_generator)
    labels[frequence] = label_b
    frequence += 1

###  Affichage d'une matrice de confusion :

In [ ]:
label_index = np.argmax(labels, axis=1)
pred_index = np.argmax(predictions, axis=1)

np.set_printoptions(precision=2)

In [ ]:
# Plot non-normalized confusion matrix
_ = afficher_matrice_confusion(label_index,
                               pred_index,
                               classes=np.array(list(classes_couverture)),
                               dictionnaire_classes=classes_couverture)


In [ ]:
# Plot normalized confusion matrix
_ = afficher_matrice_confusion(label_index,
                               pred_index,
                               classes=np.array(list(classes_couverture)),
                               dictionnaire_classes=classes_couverture,
                               normaliser=True)


In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score
print("Exactitude: {exactitude:.2f}%".format(exactitude = accuracy_score(label_index, pred_index)*100))
print("Précision: {precision:.2f}%".format(precision = precision_score(label_index, pred_index, average='weighted')*100))
print("Rappel: {rappel:.2f}%".format(rappel = recall_score(label_index, pred_index, average='weighted')*100))
print("Métrique F1: {f1:.2f}%".format(f1 = f1_score(label_index, pred_index, average='weighted')*100))


Pas mal! Environ 5 % d'amélioration pour un réseau convolutif par rapport à un algorithme d'apprentissage automatique classique comme la forêt aléatoire.

## Conclusion

Vous avez expérimenté avec un certain nombre de techniques d'exploration de données satellitaires et vu comment utiliser Keras pour construire un réseau convolutif profond pour une classification efficace de la couverture terrestre.

In [ ]:
print("Exécution du carnet web IPython terminée")